## Sentinel API

### Creación OAuth client

Para poder hacer uso de las APIs de Sentinel, es necesario registrarse en Copernicus Dataspace Ecosystem:

https://dataspace.copernicus.eu/

Posteriormente, es necesario crear un cliente OAuth. Para ello, hay que entrar en MY ACCOUNT > Sentinel Hub > settings. En el apartado OAuth Client, hacer click en crear y especificar un nombre para el cliente. Una vez creado, copiar el ID e ID-secreto a un documento de texto.

### Instalación

La instalación requiere una versión de python >= 3.8

In [ ]:
!pip install sentinelhub

### Librerías

In [ ]:
from sentinelhub import (CRS,
                         BBox,
                         DataCollection,
                         SHConfig,
                         MimeType,
                         MosaickingOrder,
                         SentinelHubRequest,
                         SentinelHubCatalog,
                         bbox_to_dimensions,
)

from utils import plot_image

### SHConfig

Al instalar el paquete, se debería haber creado una carpeta en el siguiente directorio:

C:/users/\<nombre_de_usuario>/.config/sentinelhub

En su interior, se encuentra el archivo config.toml, que puede abrirse con un editor de texto. Aquí tendremos que introducir nuestros datos:

```
[default-profile]

sh_client_id = '\<client_id_por_defecto>'
sh_client_secret = '\<mi_client_secret_id_por_defecto>'
sh_base_url = 'https://sh.dataspace.copernicus.eu'
sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

[myprofile]

sh_client_id = '\<mi_client_id>'
sh_client_secret = '\<mi_client_secret_id>'
sh_base_url = 'https://sh.dataspace.copernicus.eu'
sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
```

Una vez configurados los perfiles, podemos instanciar el objeto SHConfig

In [ ]:
config = SHConfig("myprofile")

### Parámetros

En los parámetros se puede especificar:
- Una geometría (BBox o Geometry): https://sentinelhub-py.readthedocs.io/en/latest/reference/sentinelhub.geometry.html#sentinelhub.geometry.BBox
- Un intervalo de tiempo
- Máxima nubosidad pérmitida

In [ ]:
# Parámetros
'''
CRS disponibles: WGS84, POP_WEB y constantes en formato UTM_<zona><dirección>
donde zona es un entero entre [1, 60] y dirección es N o S (hemisferio norte o sur)
'''

aoi_coords_wgs84 = (46.16, -16.15, 46.51, -15.58) # Formato para BBox: (min_x, min_y, max_x, max_y)
aoi_crs = CRS.WGS84 # Sistema de referencia para BBox.
resolution = 60 # Resolución del BBox
time_interval = ("2020-06-12", "2020-06-13") # Intervalo de tiempo
max_cloud_coverage = 1.0 # Máxima nubosidad permitida. Debe ser un float entre [0.0, 1.0]

In [ ]:
# Bounding box
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=aoi_crs) # Bounding Box
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
print(f"Image shape at {resolution} m resolution: {aoi_size} pixels")

### Catalog API

Para realizar consultas a la base de datos se utiliza la Catalog API. Es necesario especificar un satelite como primer parámetro en catalog.search()

In [ ]:
catalog = SentinelHubCatalog(config=config)

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=aoi_bbox,
    time=time_interval,
    fields={"include": ["id", "properties.datetime"], "exclude": []},
    filter='eo:cloud_cover < ' + str(max_cloud_coverage*100),
)
results = list(search_iterator)
print("Total number of results:", len(results))

results

### Process API

Para hacer requests, se utiliza la Process API.

#### eval script
Código en Javascript que define cómo Sentinel Hub debe procesar los datos y qué valores debe devolver. En este caso, se especifican las bandas RGB ("B02", "B03", "B04").

In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

#### instanciar SentinelHubRequest

Es necesario especificar un satélite como primer valor en el parámetro input_data. Actualmente, la colección DataCollection.SENTINEL2_L2A tiene definida un service_url antiguo (https://services.sentinel-hub.com).
Para modificar la url (https://sh.dataspace.copernicus.eu), se utiliza el método define_from().

El resultado final será un mosaico que se formará a partir de todas las imágenes obtenidas. El parámetro mosaicking_order permite definir cómo se va a generar este mosaico. En este caso, MosaickingOrder.LEAST_CC priorizará la mínima nubosidad.

In [ ]:
request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
            ),
            time_interval=time_interval,
            maxcc = max_cloud_coverage,
            mosaicking_order=MosaickingOrder.LEAST_CC
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config,
)

#### obtener y graficar los datos

In [ ]:
# Request
data_full_color = request_true_color.get_data()
# Plotting
plot_image(data_full_color[0], factor=3.5 / 255, clip_range=(0, 1))